In [6]:
import os
import pandas as pd
import torch
import re
from tqdm import tqdm
from soynlp.normalizer import repeat_normalize
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, BertTokenizer, BertForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 로드 및 전처리
file_path = './외국음식전문점.csv'
df = pd.read_csv(file_path)
df['content'] = df['content'].fillna('')
text_data = df['content'].tolist()

# 전처리 함수 정의
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^가-힣\s]', '', text)
    text = repeat_normalize(text, num_repeats=2)
    stopwords = ['이', '그', '저', '의', '을', '를', '은', '는', '에', '와', '과', '도', '으로', '그리고', '하지만', '그래서']
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text.strip()

preprocessed_data = [preprocess_text(review) for review in text_data]

# 카테고리 및 강화된 키워드 설정 ## 비추천 항목 추가
categories = ['맛', '가격', '서비스', '분위기', '추천', '비추천']
category_keywords = {
    '맛': ['맛', '요리', '음식', '메뉴', '향', '식감', '맛집', '고소', '달콤', '짭짤', '신선', '풍미', '스파이시', '향긋', '촉촉', '바삭', '탱탱', '육즙', '알싸', '상큼', '쫄깃'],
    '가격': ['가격', '비싸', '저렴', '가성비', '비용', '비교', '비율', '가격대', '할인', '구성', '합리적', '저렴하다', '가격 대비', '적당', '경제적', '비용 절감', '가치', '수익', '돈값', '고가', '저가'],
    '서비스': ['서비스', '직원', '응대', '친절', '매너', '대응', '접대', '환대', '서빙', '응대', '고객', '케어', '친절하다', '프로페셔널', '빠르다', '세심하다', '친절도', '고객 관리', '서비스 정신', '상냥', '대접'],
    '분위기': ['분위기', '환경', '인테리어', '공간', '장식', '조명', '음악', '장소', '배경', '아늑하다', '세련', '깔끔', '모던', '클래식', '고급스럽다', '편안하다', '우아', '로맨틱', '분위기 있는', '차분하다', '힐링'],
    '추천': ['추천', '소개', '다음에 또', '재방문', '다시 방문', '친구', '소개하고 싶다', '꼭 오고 싶다', '꼭 다시', '나만 알고 싶다'],
    '비추천': ['비추천', '다신 안', '절대 안', '다시 오지 않다', '다신 안 가다', '비추', '실망', '후회', '최악']
}

# 리뷰 카테고리 분류 함수
def categorize_review(review):
    tfidf_vectorizer = TfidfVectorizer()
    keyword_docs = [" ".join(keywords) for keywords in category_keywords.values()]
    tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_docs + [review])
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    top_categories = [categories[i] for i in cosine_similarities.argsort()[-2:]]
    return top_categories

# 대표 메뉴 추출
menu_keywords = ["피자", "스파게티", "버거", "스테이크", "초밥"]  # 예시 메뉴 키워드
menu_counter = Counter()
for review in preprocessed_data:
    for menu in menu_keywords:
        if menu in review:
            menu_counter[menu] += 1

top_menus = menu_counter.most_common(3)

# 감정 분석 모델 로드
tokenizer_kobert = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=3).to(device)  # 3개의 레이블: 긍정, 부정, 중립

# 감정 분석 수행 및 카테고리별 비율 분석
sentiment_results = []
category_sentiments = {category: Counter() for category in categories}

for review in tqdm(preprocessed_data, desc="Processing Reviews"):
    inputs_kobert = tokenizer_kobert(review, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        output_kobert = model_kobert(**inputs_kobert)
        sentiment = torch.argmax(output_kobert.logits, dim=1).item()

    categories_found = categorize_review(review)
    sentiment_results.append(sentiment)
    for category in categories_found:
        category_sentiments[category][sentiment] += 1

# 리뷰 요약 모델 로드 및 요약 생성 (간결한 요약 설정)
bart_tokenizer = PreTrainedTokenizerFast.from_pretrained('EbanLee/kobart-summary-v3')
bart_model = BartForConditionalGeneration.from_pretrained('EbanLee/kobart-summary-v3').to(device)

def summarize_reviews(reviews):
    combined_reviews = " ".join(reviews)
    inputs_bart = bart_tokenizer([combined_reviews], return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    summary_ids = bart_model.generate(
        inputs_bart['input_ids'], 
        attention_mask=inputs_bart['attention_mask'], 
        max_length=50,  # 더 짧은 요약
        min_length=20,  
        length_penalty=2.0,  
        num_beams=4, 
        no_repeat_ngram_size=2,  
        repetition_penalty=1.2,  
        early_stopping=True
    )
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # TextBlob을 사용하여 문장 자연스러움 개선
    blob = TextBlob(summary)
    corrected_summary = str(blob.correct())
    
    # 'of' 같은 불필요한 단어 제거
    corrected_summary = re.sub(r'\bof\b', '', corrected_summary)
    
    return corrected_summary

# 가게별 리뷰 종합 요약 및 점수 계산

# 그룹화된 데이터를 가게별로 가져오기
store_grouped = df.groupby('store_name')['content']

store_summaries = []

# 순차적으로 가게별 리뷰를 요약 생성 및 점수 계산
for store_name, reviews in tqdm(store_grouped, desc="Processing Stores"):
    try:
        reviews = reviews.tolist()  # 그룹화된 데이터는 시리즈로 나오므로 리스트로 변환
        preprocessed_reviews = [preprocess_text(review) for review in reviews]
        summary = summarize_reviews(preprocessed_reviews)

        # 감성 비율 계산
        sentiments = [sentiment_results[i] for i, r in enumerate(preprocessed_data) if r in reviews]
        sentiment_counts = Counter(sentiments)
        total_reviews = len(sentiments)
        sentiment_ratios = {label: sentiment_counts.get(i, 0) / total_reviews for i, label in enumerate(['Negative', 'Neutral', 'Positive'])}

        # 카테고리별 점수 계산
        category_scores = {}
        for category in categories:
            category_count = sum(category_sentiments[category].values())
            category_scores[category] = category_sentiments[category][2] / category_count if category_count > 0 else 0  # 긍정 점수

        # 최종 점수 계산 (예: 감성 비율과 카테고리 점수의 평균)
        overall_score = (sentiment_ratios['Positive'] + sum(category_scores.values()) / len(categories)) / 2

        # 결과 저장
        store_summaries.append({
            'Store Name': store_name,
            'Summary': summary,
            'Negative Ratio': sentiment_ratios['Negative'],
            'Neutral Ratio': sentiment_ratios['Neutral'],
            'Positive Ratio': sentiment_ratios['Positive'],
            'Taste Score': category_scores['맛'],
            'Price Score': category_scores['가격'],
            'Service Score': category_scores['서비스'],
            'Atmosphere Score': category_scores['분위기'],
            'Recommendation Score': category_scores['추천'],
            'Non-recommendation Score': category_scores['비추천'],
            'Overall Score': overall_score
        })

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")

# 현재 날짜와 시간을 가져와서 파일 이름에 포함
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}_리뷰요약및점수.csv'

# 결과를 CSV 파일로 저장
try:
    final_results = pd.DataFrame(store_summaries)
    final_results.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")
except Exception as e:
    print(f"Error in saving results to CSV: {e}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
C:\Users\G-01\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing

Error processing store 남미 플랜트 랩: division by zero


Processing Stores:  10%|█         | 76/737 [03:46<31:41,  2.88s/it]

Error processing store 다올: division by zero


Processing Stores:  13%|█▎        | 93/737 [04:41<33:08,  3.09s/it]

...

Error processing store 핸드파스타(Hand pasta): division by zero


Processing Stores:  98%|█████████▊| 723/737 [36:01<00:41,  2.98s/it]

Error processing store 후안타이: division by zero


Processing Stores: 100%|██████████| 737/737 [36:41<00:00,  2.99s/it]

Results saved to 20240901_170257_리뷰요약및점수.csv


In [9]:
import pandas as pd

# 데이터 불러오기
file_path = './20240901_170257_리뷰요약및점수.csv'  # 실제 파일 경로로 변경하세요
df = pd.read_csv(file_path)

# 가중치 설정
weights = {
    'Positive Ratio': 50,
    'Neutral Ratio': 20,
    'Negative Ratio': -30,
    'Recommendation Score': 30,
    'Non-recommendation Score': -30
}

# 각 가게의 최종 점수 계산 (100점 만점)
df['Final Score'] = (
    df['Positive Ratio'] * weights['Positive Ratio'] +
    df['Neutral Ratio'] * weights['Neutral Ratio'] +
    df['Negative Ratio'] * weights['Negative Ratio'] +
    df['Recommendation Score'] * weights['Recommendation Score'] +
    df['Non-recommendation Score'] * weights['Non-recommendation Score']
)

# 100점 만점으로 정규화
df['Final Score'] = df['Final Score'].clip(0, 100)  # 점수가 0보다 작거나 100을 넘지 않도록 조정

# 상위 10개 가게 선택
top_10_stores = df.sort_values(by='Final Score', ascending=False).head(10)

# 최종 점수 출력
print(top_10_stores[['Store Name', 'Final Score']])


               Store Name  Final Score
249            발리다포차(한남점)    50.151089
262   버팔로스팟(Buffalo Spot)    50.137258
0                 (주)애브릿지    50.137232
707                 흑석호치민    50.137232
381             아토키토 보라매점    50.137232
136      로찌마곡(LodgeMagok)    50.137232
24              강남쌀국수 마곡점    50.137232
541                퀼 QUIL    50.123388
422                   오빈스    50.109557
523  커피버넷(COFFEE BURNETT)    50.109557
